In [53]:
!pip install requests beautifulsoup4 pandas openpyxl lxml

In [54]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import itertools

#########################
# Utility functions
#########################
def flatten_multiindex_columns(df):
    """
    Given a DataFrame with a MultiIndex for columns (possibly 3 levels, e.g.):
      (Table Title, Group, Detail)
    this function will:
      1. If the first level is identical across all columns (e.g. "Drivers (DV)"),
         drop that level.
      2. For the remaining tuple (usually 2-level), if both parts are identical,
         return just one part; otherwise, join them with " - ".
    """
    if isinstance(df.columns, pd.MultiIndex):
        first_level = df.columns.get_level_values(0)
        if len(set(first_level)) == 1:
            new_cols = []
            for col in df.columns:
                sub_tuple = col[1:]
                parts = [str(x).strip() for x in sub_tuple if x and not str(x).lower().startswith("unnamed")]
                if not parts:
                    new_cols.append("")
                elif len(parts) == 1:
                    new_cols.append(parts[0])
                else:
                    if parts[0] == parts[1]:
                        new_cols.append(parts[0])
                    else:
                        new_cols.append(" - ".join(parts))
            df.columns = new_cols
        else:
            new_cols = []
            for col in df.columns:
                parts = [str(x).strip() for x in col if x and not str(x).lower().startswith("unnamed")]
                new_cols.append(" - ".join(parts))
            df.columns = new_cols
    return df

def replace_img_with_alt(table):
    """
    In the given BeautifulSoup table element, find all <td> and <th> cells.
    If a cell contains one or more <img> tags, replace the cell’s contents with
    a comma-separated string of the images’ alt attribute values.
    """
    for cell in table.find_all(["td", "th"]):
        imgs = cell.find_all("img")
        if imgs:
            alt_texts = [img.get("alt", "").strip() for img in imgs if img.get("alt")]
            if alt_texts:
                cell.clear()
                cell.append(", ".join(alt_texts))
    return table

def remove_footer(table):
    """
    Remove any <tfoot> element from the BeautifulSoup table.
    """
    for t in table.find_all("tfoot"):
        t.extract()
    return table

def drop_footer_row(df):
    """
    If the last row in the DataFrame contains footer text (like "Unused statistic")
    in every cell, drop that row.
    """
    if not df.empty:
        last_row = df.iloc[-1]
        if all("Unused statistic" in str(val) for val in last_row):
            df = df.iloc[:-1]
    return df

def separate_numeric_and_nonnumeric(df, identifier):
    """
    For the given DataFrame, separate columns (other than the identifier column)
    into numeric and non-numeric.
    
    Returns two dictionaries mapping column names to Series.
      - If pd.to_numeric() converts all values in a column to NaN, it is treated as non-numeric.
    """
    numeric_cols = {}
    nonnumeric_cols = {}
    for col in df.columns:
        if col == identifier:
            continue
        converted = pd.to_numeric(df[col], errors="coerce")
        # If no value could be converted, treat the column as non-numeric.
        if converted.notna().sum() == 0:
            nonnumeric_cols[col] = df[col]
        else:
            # Use the converted column; fill NaN with 0.
            numeric_cols[col] = converted.fillna(0)
    return numeric_cols, nonnumeric_cols

#########################
# Table Extraction
#########################
def extract_tables(url, table_labels):
    """
    Fetches the page at `url`, replaces image tags with alt text, removes any table footer,
    and then extracts tables whose title (a <th> with colspan in the <thead>)
    contains one of the strings in `table_labels`.
    
    For tables with at least 3 header rows (the first being the table title),
    we use pd.read_html with header=[0,1,2] so that the columns come in as a MultiIndex.
    We then flatten the columns.
    
    Returns a dictionary mapping each label to its corresponding DataFrame.
    """
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    tables_dict = {}
    
    for table in soup.find_all("table", class_="sortable"):
        title_th = table.find("th", colspan=True)
        if title_th:
            title_text = title_th.get_text(strip=True)
            for label in table_labels:
                if label in title_text:
                    table = remove_footer(table)
                    table = replace_img_with_alt(table)
                    
                    thead = table.find("thead")
                    header_rows = thead.find_all("tr") if thead else []
                    if len(header_rows) >= 3:
                        df_list = pd.read_html(str(table), header=[0, 1, 2])
                    elif len(header_rows) == 2:
                        df_list = pd.read_html(str(table), header=[0, 1])
                    else:
                        df_list = pd.read_html(str(table))
                    
                    if df_list:
                        df = df_list[0]
                        df = flatten_multiindex_columns(df)
                        df = drop_footer_row(df)
                        tables_dict[label] = df
    return tables_dict

#########################
# Build Table Computation
#########################
def compute_build_table(driver_df, body_df, tire_df, glider_df):
    """
    Computes a new DataFrame containing every possible build – one row from each component table –
    and sums the numeric stat values across the four parts.
    
    For non-numeric stat columns (e.g. "Size" or "Vehicle Type"), which are assumed to appear in only one table,
    the value is carried over from that table.
    
    Identification:
      - For drivers, the identifier is the "Character" column.
      - For bodies, tires, and gliders, the identifier is taken from the first column.
    """
    # Identify the component IDs.
    driver_id = driver_df["Character"]
    body_id   = body_df.iloc[:, 0]
    tire_id   = tire_df.iloc[:, 0]
    glider_id = glider_df.iloc[:, 0]
    
    # Separate each table's stat columns into numeric and non-numeric.
    driver_numeric, driver_nonnum = separate_numeric_and_nonnumeric(driver_df, "Character")
    body_numeric,   body_nonnum   = separate_numeric_and_nonnumeric(body_df, body_df.columns[0])
    tire_numeric,   tire_nonnum   = separate_numeric_and_nonnumeric(tire_df, tire_df.columns[0])
    glider_numeric, glider_nonnum = separate_numeric_and_nonnumeric(glider_df, glider_df.columns[0])
    
    # Union of numeric stat column names across tables.
    numeric_stats = set(driver_numeric.keys()) | set(body_numeric.keys()) | set(tire_numeric.keys()) | set(glider_numeric.keys())
    numeric_stats = sorted(list(numeric_stats))
    
    # For non-numeric stats, assume that they appear only in one table.
    nonnumeric_stats = {}
    for col, series in driver_nonnum.items():
        nonnumeric_stats[col] = ("Driver", series)
    for col, series in body_nonnum.items():
        nonnumeric_stats[col] = ("Body", series)
    for col, series in tire_nonnum.items():
        nonnumeric_stats[col] = ("Tire", series)
    for col, series in glider_nonnum.items():
        nonnumeric_stats[col] = ("Glider", series)
    
    build_rows = []
    # Iterate over every combination (Cartesian product).
    for d_idx, b_idx, t_idx, g_idx in itertools.product(driver_df.index, body_df.index, tire_df.index, glider_df.index):
        row = {}
        row["Driver"] = driver_id.loc[d_idx]
        row["Body"]   = body_id.loc[b_idx]
        row["Tire"]   = tire_id.loc[t_idx]
        row["Glider"] = glider_id.loc[g_idx]
        
        # For each numeric stat, sum its values from the four tables (using 0 if absent).
        for stat in numeric_stats:
            total = 0
            if stat in driver_numeric:
                total += driver_numeric[stat].loc[d_idx]
            if stat in body_numeric:
                total += body_numeric[stat].loc[b_idx]
            if stat in tire_numeric:
                total += tire_numeric[stat].loc[t_idx]
            if stat in glider_numeric:
                total += glider_numeric[stat].loc[g_idx]
            row[stat] = total
        
        # For each non-numeric stat, simply use the value from the table that contains it.
        for col, (source, series) in nonnumeric_stats.items():
            if source == "Driver":
                row[col] = series.loc[d_idx]
            elif source == "Body":
                row[col] = series.loc[b_idx]
            elif source == "Tire":
                row[col] = series.loc[t_idx]
            elif source == "Glider":
                row[col] = series.loc[g_idx]
        
        build_rows.append(row)
    build_df = pd.DataFrame(build_rows)
    return build_df

#########################
# Save to Excel
#########################
def save_tables_to_excel(tables_dict, output_file):
    """
    Saves each DataFrame in `tables_dict` to a separate sheet in an Excel workbook.
    """
    with pd.ExcelWriter(output_file) as writer:
        for label, df in tables_dict.items():
            sheet_name = label if len(label) <= 31 else label[:31]
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    print(f"Data saved to {output_file}")

#########################
# Main Execution
#########################
if __name__ == "__main__":
    url = "https://www.mariowiki.com/Mario_Kart_8_Deluxe_in-game_statistics"
    table_labels = ["Drivers (DV)", "Bodies (BD)", "Tires (TR)", "Gliders (WG)"]
    
    tables = extract_tables(url, table_labels)
    if not tables:
        print("No matching tables found on the page.")
    else:
        try:
            driver_df = tables["Drivers (DV)"]
            body_df   = tables["Bodies (BD)"]
            tire_df   = tables["Tires (TR)"]
            glider_df = tables["Gliders (WG)"]
            
            print("Computing build combinations... (this may take a while if there are many rows)")
            build_df = compute_build_table(driver_df, body_df, tire_df, glider_df)
            tables["Builds"] = build_df
        except KeyError as e:
            print(f"Missing one of the required tables: {e}")
        
        save_tables_to_excel(tables, "mario_kart_stats.xlsx")


/tmp/ipykernel_3952/4130451171.py:136: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_list = pd.read_html(str(table))
/tmp/ipykernel_3952/4130451171.py:136: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_list = pd.read_html(str(table))
/tmp/ipykernel_3952/4130451171.py:136: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_list = pd.read_html(str(table))
/tmp/ipykernel_3952/4130451171.py:136: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_list = pd.read_html(str(table))


Computing build combinations... (this may take a while if there are many rows)
Data saved to mario_kart_stats.xlsx
